In [ ]:
{
 "cells": [
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# ANI pipeline con skani y fastANI\n",
    "Descarga genomas, descomprime, genera listas y calcula ANI.\n"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "!echo \"Ingresa taxID\"\n",
    "taxid = input()\n",
    "output_dir = f\"genomes_tax_{taxid}\"\n"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "!mkdir -p \"$output_dir\"\n",
    "SCINAME=$(esearch -db taxonomy -query \"$taxid\" | efetch -format xml | xtract -pattern Taxon -element ScientificName)\n",
    "if [[ -z \"$SCINAME\" ]]; then\n",
    "    print('TaxID inválido o sin resultados')\n",
    "    exit(1)\n",
    "else:\n",
    "    print(f\"Organismo detectado: {SCINAME}\")\n",
    "!\n",
    "ncbi-genome-download bacteria \\\n",
    "    --genera \"$SCINAME\" \\\n",
    "    --assembly-level complete \\\n",
    "    --formats fasta \\\n",
    "    --section refseq \\\n",
    "    --output-folder \"$output_dir\" \\\n",
    "    --flat-output\n"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "!find \"$output_dir\" -type f -name \"*.fna.gz\" -exec gunzip -f {} \\;"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "!find \"$output_dir\" -type f -name \"*.fna\" > genome_list.txt\n",
    "!wc -l genome_list.txt\n",
    "!head genome_list.txt"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "!skani triangle -l genome_list.txt -o skani_output.tsv -t 2"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "!cp genome_list.txt fastani_query_list.txt\n",
    "!cp genome_list.txt fastani_ref_list.txt\n",
    "!fastANI --ql fastani_query_list.txt --rl fastani_ref_list.txt --fragLen 3000 -t 2 -o fastani_output.tsv"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "import pandas as pd\n",
    "df_skani = pd.read_csv(\"skani_output.tsv\", sep=\"\\t\")\n",
    "df_fastani = pd.read_csv(\"fastani_output.tsv\", sep=\"\\t\", header=None,\n",
    "                         names=[\"Query\", \"Reference\", \"ANI\", \"Fragments\", \"Total_Frags\"])\n",
    "print(\"SKANI ANI:\")\n",
    "display(df_skani.head())\n",
    "print(\"FASTANI ANI:\")\n",
    "display(df_fastani.head())"
   ]
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": "Python 3",
   "language": "python",
   "name": "python3"
  },
  "language_info": {
   "codemirror_mode": {
    "name": "ipython",
    "version": 3
   },
   "file_extension": ".py",
   "mimetype": "text/x-python",
   "name": "python",
   "nbconvert_exporter": "python",
   "pygments_lexer": "ipython3",
   "version": "3.10.8"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 5
}

